<a href="https://colab.research.google.com/github/ProsperChuks/Population-Estimate-and-Projection/blob/main/forecasts/forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [6]:
!pip install statsmodels --upgrade

     |████████████████████████████████| 9.8 MB 5.1 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [5]:
import pickle
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.express as px

## Loading Models

In [2]:
cdr_forecast = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/Hamoye Intership/Capstone Project/models/cdr_model.pkl', 'rb'))

## Forecasting Crude Death Rate(Per 1000 Persons) up to Year 2040

In [13]:
predict = cdr_forecast.forecast(500)

fig = px.line(predict, predict.index, 0, 
        labels={'x':'Year', '0': 'CDR (%)'})

fig.update_layout(plot_bgcolor='pink')